<a href="https://colab.research.google.com/github/Inishpy/DeepLearningWork/blob/main/financeModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
from huggingface_hub import hf_hub_download
torch.set_default_device("cuda")

In [ ]:
def print_with_newline(string):
    words = string.split()
    for i in range(0, len(words), 10):
        print(' '.join(words[i:i+10]))

In [ ]:
pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 28.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.46-cp310-cp310-manylinux_2_35_x86_64.whl size=2615886 sha256=2bb0415dc6cce667d4ec166559b583bff63a7ed6b4781215b25deb4ab07e9f04
  Stored in directory: /root/.cache/pip/wheels/1d/63/cb/c1ce62664712ab6fd32f32b9e8015bc85e6b335477471be3e4
Successfully built llama-cpp-python


In [ ]:
from llama_cpp import Llama

In [ ]:
#FINANCE MODELS
#model1 = hf_hub_download(repo_id="TheBloke/finance-LLM-GGUF", filename="finance-llm.Q2_K.gguf")
#model2 = hf_hub_download(repo_id="TheBloke/finance-LLM-GGUF", filename="finance-llm.Q4_0.gguf")
#model3 = hf_hub_download(repo_id="TheBloke/finance-LLM-GGUF", filename="finance-llm.Q5_K_S.gguf")
#model4 = hf_hub_download(repo_id="TheBloke/finance-LLM-13B-GGUF", filename="finance-llm-13b.Q4_K_M.gguf")
#model4 = hf_hub_download(repo_id="TheBloke/finance-LLM-13B-GGUF", filename="finance-llm-13b.Q5_K_M.gguf")
#model4 = hf_hub_download(repo_id="TheBloke/finance-chat-GGUF", filename="finance-chat.Q5_K_M.gguf")

#PHI VERSIONS
modelphi2 = hf_hub_download(repo_id="TheBloke/phi-2-dpo-GGUF", filename="phi-2-dpo.Q5_K_S.gguf")
#modelphi2_ = hf_hub_download(repo_id="TheBloke/phi-2-dpo-GGUF", filename="phi-2-dpo.Q8_0.gguf")
#modelphi0 = hf_hub_download(repo_id="TheBloke/phi-2-GGUF", filename="phi-2.Q5_K_M.gguf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


phi-2-dpo.Q5_K_S.gguf:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

In [ ]:
#llmModelphiOriginal = Llama(model_path= modelphi0, chat_format="llama-2",  n_gpu_layers=30)
llmModelphi2dpoq5 = Llama(model_path= modelphi2, chat_format="llama-2",   n_gpu_layers=30)
#llmModelphi2dpoq8 = Llama(model_path= modelphi2_, chat_format="llama-2",   n_gpu_layers=30)



llama_model_loader: loaded meta data with 21 key-value pairs and 325 tensors from /root/.cache/huggingface/hub/models--TheBloke--phi-2-dpo-GGUF/snapshots/807739a7ff89883d9a736ac82b276926a79f4ac6/phi-2-dpo.Q5_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi2
llama_model_loader: - kv   1:                               general.name str              = Phi2
llama_model_loader: - kv   2:                        phi2.context_length u32              = 2048
llama_model_loader: - kv   3:                      phi2.embedding_length u32              = 2560
llama_model_loader: - kv   4:                   phi2.feed_forward_length u32              = 10240
llama_model_loader: - kv   5:                           phi2.block_count u32              = 32
llama_model_loader: - kv   6:                  phi2.attention.head_count u32 

In [ ]:

llmModelphiOriginal.token_eos()

50256

In [ ]:

def Model(llmModel, input):
  start_time = time.time()

  output = ""
  #llm = Llama(model_path= model, chat_format="llama-2", n_ctx=2048,  n_gpu_layers=30)  # Set chat_format according to the model you are using
  for token in  (llmModel.create_chat_completion(
      messages = [

          {
              "role": "user",
              "content": "Instruct: {instruction} . Answer this question in 6 sentences.\nOutput:".format(instruction = input)
          }
      ],
      stream = True,
      max_tokens = None,
      stop = ["Instruct:", "Output:", "[Inst]"]

  )):
    output += token['choices'][0]['delta'].get('content', "")
    print(token['choices'][0]['delta'].get('content', ""))


  end_time = time.time()
  print("------------------------------------------")

  print_with_newline(output)
  print("------------------------------------------")
  #print(f"{output['usage']['completion_tokens']/(end_time - start_time)} tokens per seconds")


def Model_(model,input):
  llm = Llama(
  model_path=model,  # Download the model file first
  n_ctx=2048,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
  )

  # Simple inference example
  output = llm(
      "### Human: {input}\n\n### Assistant: ".format(input=input),
    #"### Instruction: {instruction}\n\n### Input: {input} \n\n### Answer:".format(instruction=instruction, input=input), # Prompt
    max_tokens=512,  # Generate up to 512 tokens
    # Example stop token - not necessarily correct for this specific model! Please check before using.
    echo=True        # Whether to echo the prompt
  )



In [ ]:
prompt = "Input: what is the name of the first president of India?  always end the answer with full stop \n Output: "

In [ ]:
prompt1 = "what is the name of the first president of India? "

In [ ]:
Model(llmModelphiOriginal, prompt1)

In [ ]:
Model(llmModelphi2dpoq5, prompt1)

Llama.generate: prefix-match hit



The
 first
 President
 of
 India
 was
 Dr
.
 Raj
endra
 Pr
as
ad
,
 who
 served
 from
 1950
 to
 1956
.
 He
 was
 a
 prominent
 leader
 of
 the
 Indian
 National
 Congress
 and
 a
 freedom
 fighter
 during
 the
 struggle
 for
 independence
 from
 British
 rule
.
 He
 was
 also
 a
 scholar
 and
 a
 states
man
,
 having
 served
 as
 the
 first
 Chief
 Minister
 of
 Bihar
 and
 as
 the
 Minister
 for
 Education
 and
 Law
 and
 Justice
 in
 the
 Const
itu
ent
 Assembly
.




Dr
.
 Raj
endra
 Pr
as
ad
 was
 elected
 as
 the
 President
 of
 India
 by
 an
 electoral
 college
 consisting
 of
 members
 of
 both
 houses
 of
 Parliament
 and
 state
 legislatures
.
 He
 took
 oath
 as
 the
 President
 on
 26
 January
 1950
 at
 Rash
trap
ati
 Bh
avan
,
 the
 official
 residence
 of
 the
 President
 of
 India
.
 He
 continued
 as
 the
 President
 until
 his
 death
 on
 24
 January
 1956
,
 after
 completing
 two
 full
 five
-
year
 terms
.




During
 his
 tenure
 as
 the
 President
,
 Dr
.
 Raj
e


llama_print_timings:        load time =   10082.84 ms
llama_print_timings:      sample time =     405.64 ms /   400 runs   (    1.01 ms per token,   986.10 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  133728.33 ms /   400 runs   (  334.32 ms per token,     2.99 tokens per second)
llama_print_timings:       total time =  136782.53 ms /   401 tokens



------------------------------------------
The first President of India was Dr. Rajendra Prasad, who
served from 1950 to 1956. He was a prominent leader
of the Indian National Congress and a freedom fighter during
the struggle for independence from British rule. He was also
a scholar and a statesman, having served as the first
Chief Minister of Bihar and as the Minister for Education
and Law and Justice in the Constituent Assembly. Dr. Rajendra
Prasad was elected as the President of India by an
electoral college consisting of members of both houses of Parliament
and state legislatures. He took oath as the President on
26 January 1950 at Rashtrapati Bhavan, the official residence of
the President of India. He continued as the President until
his death on 24 January 1956, after completing two full
five-year terms. During his tenure as the President, Dr. Rajendra
Prasad played a significant role in shaping the early years
of independent India. He presided over the first session of
the Co

In [ ]:
Model(llmModelphi2dpoq8, prompt1)

Llama.generate: prefix-match hit





[
INS
ERT
_
NAME
_
OF
_
FIR
ST
_
PRES
IDENT
_
OF
_
IND
IA
]


<
[/
INS
ERT
_
NAME
_
OF
_
FIR
ST
_
PRES
IDENT
_
OF
_
IND
IA
]]


Hello
!


KeyboardInterrupt: 

In [ ]:
Model(model3, input)

In [ ]:
Model(model4, input)

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp

In [ ]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00


In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=modelphi2,
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

In [ ]:
template = """Input: {input}

Instruction: What is the sentiment of this news? Please choose an answer from negative,neutral or positive.

Answer: understand the Instruction and answer accordingly about the given Input"""

prompt = PromptTemplate.from_template(template)




In [ ]:
template = """Input: {input}

Instruction: Answer consisely like a chatbot.

Answer: understand the Instruction and answer accordingly about the given Input"""

prompt = PromptTemplate.from_template(template)




In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
input = "Glaxo's ViiV Healthcare Signs China Manufacturing Deal With Desano"
input = "Who is the first president of India?"
llm_chain.run(input)

In [ ]:
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters
llm = ChatOllama(model="llama2")
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# using LangChain Expressive Language chain syntax
# learn more about the LCEL on
# https://python.langchain.com/docs/expression_language/why
chain = prompt | llm | StrOutputParser()

# for brevity, response is printed in terminal
# You can use LangServe to deploy your application for
# production
print(chain.invoke({"topic": "Space travel"}))

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7840c59ced40>: Failed to establish a new connection: [Errno 111] Connection refused'))